# Models need to adhere to the proportional hazards assumption
Let's see if ours do!

| |Index events defining group of interest|Index events defining comparison group|Excluded|
|-|-------------|---------------|-------------|
| Grouping 1 |Abandoned 111 calls with triaged call in previous 72 hours|Triaged 111 calls with no abandoned call in following 72 hours|**+** Abandoned call without triaged 111 call in previous 72 hours **+** Triaged or abandoned 111 calls occurring in follow-up for index events (these are outcomes)|
|Grouping 2|Abandoned 111 calls without triaged 111 call in previous 72 hours|Triaged 111 calls|Triaged or abandoned 111 calls occurring in follow-up for index events (these are outcomes)|
|Grouping 3|Abandoned 111 calls with or without triaged 111 call in previous 72 hours|Triaged 111 calls with no abandoned call in following 72 hours|Triaged or abandoned 111 calls occurring in follow-up for index events (these are outcomes)|

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(lubridate))
suppressPackageStartupMessages(library(glue))
suppressPackageStartupMessages(library(survival))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggfortify))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(ggsurvfit))
suppressPackageStartupMessages(library(tidycmprsk))
suppressPackageStartupMessages(library(survminer))
suppressPackageStartupMessages(library(gtsummary))
suppressPackageStartupMessages(library(flextable))
suppressPackageStartupMessages(library(coxme))
suppressPackageStartupMessages(library(flextable))
suppressPackageStartupMessages(library(locfit))

In [ ]:
options(repr.plot.width=14, repr.plot.height=6)

In [ ]:
grouping_number = 3

follow_up_time = 72

follow_up_time_str = ""
if(follow_up_time != 72) {
 follow_up_time_str <- glue::glue("_{follow_up_time}")   
}

In [ ]:
grouping_df <- readRDS(glue::glue('data/grouping{grouping_number}_survival_df{follow_up_time_str}.rds')) %>%
    filter(!is.na(age) & age >= 18) # ADults only 2024-04-19

In [ ]:
imd_ethnicity_fn <- function(df) {
 df %>%
    mutate(
        person_id = bit64::as.integer.integer64(person_id),
        imd_quintile = as.factor(case_when(
            bit64::as.integer.integer64(imd_decile) %in% c(1, 2) ~ 1,
            bit64::as.integer.integer64(imd_decile) %in% c(3, 4) ~ 2,
            bit64::as.integer.integer64(imd_decile) %in% c(5, 6) ~ 3,
            bit64::as.integer.integer64(imd_decile) %in% c(7, 8) ~ 4,
            bit64::as.integer.integer64(imd_decile) %in% c(9, 10) ~ 5,
            TRUE ~ NA_integer_
        )),
        # Small number of cases where birthdate calculation results in age being -1
        age = case_when(
            age < 0 ~ 0.0,
            age > 107 ~ NA_real_,
            TRUE ~ as.numeric(as.character(age))
        ),
        age_cat = case_when(
            between(age, 18, 64) ~ "18-64",
            age > 64 ~ "over 65"
        ),
        ethnicity_simple = fct_relevel(as.factor(str_extract(ethnicity_source_value, "[^:]+")), 'White'),
        gp_visit_non_avoid = as.factor(if_else(!is.na(num_GP_contacts_to_ED_non_avoid_attend), "gp_visit", "no_gp_visit")),
        gp_visit = as.factor(if_else(!is.na(num_GP_contacts_to_ED_attend), "gp_visit", "no_gp_visit")),
    ) %>% filter(!is.na(age))
}

In [ ]:
groupinga_df <- imd_ethnicity_fn(grouping_df)

In [ ]:
#groupinga_df %>% glimpse()

# Kaplan-Meier plots
Simple model looking at cohort only

In [ ]:
# https://www.danieldsjoberg.com/ggsurvfit/reference/ggsurvfit.html
km_plot_fn <- function(df, miny = 0, maxy = 0.99, grouping_num = 1, legend_pos = 'LOW', covars = "cohort", non_avoid_var = "_non_avoid", type="km") {
    
    legend_pos_y = 0.25
    if(legend_pos != 'LOW') {
        legend_pos_y = 0.75
    }
    
    km_formula <- glue::glue("Surv(fu_time{non_avoid_var}, status{non_avoid_var}) ~ {covars}")
    
    km_fit <- survfit2(as.formula(km_formula), data = df)
    
    if(type == "cloglog") {
        # Inspired by: https://grodri.github.io/survival/cox
        dkm <- tibble(
            time = km_fit$time, 
            survival = km_fit$surv,
            group = factor(rep(names(km_fit$strata), km_fit$strata)),
            lls = log(-log(survival))
        )
        
        plot <- dkm %>%
            ggplot(aes(x = log(time), y = lls, color=group)) + 
                #geom_point() +
                geom_line() + 
                ylab("log(-log(S(t)))") +
                theme_minimal() +
                theme(plot.margin = margin(0, 0, 50, 0, "pt"),plot.title=element_text(face="bold")) +
                ggtitle(glue::glue("Log-log plot for {covars} {if_else(non_avoid_var == '_non_avoid', 'non_avoidable ED attendance', 'All ED attendance')} grouping {grouping_number}"))
                #theme(legend.position = "bottom")
    } else if(type == "ci") {
        cum_df <- df %>% mutate(
            fu_time,
            fu_time_non_avoid,
            status = as.factor(status),
            status_non_avoid = as.factor(status_non_avoid)
        )
        plot <- cuminc(as.formula(km_formula), data = cum_df) %>% 
          ggcuminc() +
          labs(
            title = glue::glue("Cumulative incidence of {if_else(non_avoid_var == '_non_avoid', 'non-avoidable ED attendance', 'All ED attendance')} grouping {grouping_number}"),
            x = "Hours",
            y = "Cumulative Incidence"
            ) + 
          scale_y_continuous(limits = c(miny, maxy), label = scales::label_percent()) +
          scale_x_continuous(limits = c(0, 72), label = seq(0, 72, 12), breaks = seq(0, 72, 12)) +
          add_confidence_interval() +
          #add_risktable() +
          theme(legend.position.inside = c(0.85, legend_pos_y)) 
    } else {
        plot <- km_fit %>% 
          ggsurvfit() +
          labs(
            title = glue::glue("{if_else(non_avoid_var == '_non_avoid', 'Non-avoidable ED attendances', 'All ED attendances')} grouping {grouping_number}"),
            x = "Hours",
            y = "Probability of NOT attending ED"
            ) + 
          scale_y_continuous(limits = c(miny, maxy), label = scales::label_percent()) +
          scale_x_continuous(limits = c(0, 72), label = seq(0, 72, 12), breaks = seq(0, 72, 12)) +
          add_confidence_interval() +
          #add_risktable() +
          theme(legend.position.inside = c(0.85, legend_pos_y)) 
    }
    
    return(list(plot, km_fit))
    
}

In [ ]:
cum_incidence <- km_list_non_avoid <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number, type="ci")
cum_incidence_all <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number, non_avoid = "", type="ci")
final_plot_ci <- patchwork::wrap_plots(wrap_elements(cum_incidence[[1]]), wrap_elements(cum_incidence_all[[1]]), ncol = 2)
ggsave(plot = final_plot_ci, file = glue::glue("output/ci_plots_grouping{grouping_number}.pdf"), width = 12)
final_plot_ci

In [ ]:
km_list_non_avoid <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number)
saveRDS(km_list_non_avoid[[1]], glue::glue("output/km_non_avoid_plot_grouping{grouping_number}.rds"))
km_list_all <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number, non_avoid = "")
final_plot <- patchwork::wrap_plots(wrap_elements(km_list_non_avoid[[1]]), wrap_elements(km_list_all[[1]]), ncol = 2)
ggsave(plot = final_plot, file = glue::glue("output/km_plots_grouping{grouping_number}.pdf"), width = 12)
final_plot

In [ ]:
km_surv_diff <- survdiff(Surv(fu_time_non_avoid, status_non_avoid) ~ cohort, data = groupinga_df)

In [ ]:
grouping_df %>% filter(status_non_avoid == 1) %>% 
    group_by(cohort) %>%
    summarise(
        med = median(fu_time_non_avoid, na.rm = T)
    ) %>% ungroup()

In [ ]:
grouping_df %>% filter(status_non_avoid == 1) %>% 
    count(cohort, fu_time_non_avoid) %>%
    ggplot(aes(x = fu_time_non_avoid, y =n, fill = cohort)) +
    geom_col() +
    facet_wrap(~cohort)

In [ ]:
km_surv_diff

# Log-log plot

From: https://bookdown.org/sestelo/sa_financial/how-to-evaluate-the-ph-assumption.html

This expression indicates that if we use a Cox model (well-used) and plot the estimated log-log survival curves for individuals on the same graph, the two plots would be approximately parallel. The distance between the two curves is the linear expression involving the differences in predictor values, which does not involve time.

Note that there is an important problem associated with this approach, that is, how to decide “how parallel is parallel?”. This fact can be subjective, thus the proposal is to be conservative for this decision by assuming the PH assumption is satisfied unless there is strong evidence of nonparallelism of the log–log curves.

In [ ]:
km_list_non_avoid <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number, type = "cloglog")
km_list_all <- km_plot_fn(df = groupinga_df, grouping_num = grouping_number, non_avoid = "", type = "cloglog")
final_plot_ll <- patchwork::wrap_plots(wrap_elements(km_list_non_avoid[[1]]), wrap_elements(km_list_all[[1]]), ncol = 2)
final_plot_ll

In [ ]:
cox_fn <- function(
    df, 
    grouping = 1, 
    non_avoid = FALSE, 
    basic = FALSE,
    basic_covar = "cohort",
    null_model = FALSE,
    custom_formula = ""
) {
 

    non_avoid_var <- if_else(non_avoid == TRUE, "_non_avoid", "")

    cox_formula = glue::glue("Surv(fu_time{non_avoid_var}, status{non_avoid_var}) ~ {basic_covar}")
    if(basic == FALSE) {
        cox_formula = glue::glue("Surv(fu_time{non_avoid_var}, status{non_avoid_var}) ~ cohort + age + sex + imd_quintile + ethnicity_simple + ooh + gp_visit{non_avoid_var}")
        if(custom_formula != "") {
            cox_formula = glue::glue("Surv(fu_time{non_avoid_var}, status{non_avoid_var}) ~ {custom_formula}")
        }
    } 
    if(null_model == TRUE) {
        cox_formula = glue::glue("Surv(fu_time{non_avoid_var}, status{non_avoid_var}) ~ 1")
    }
    print(glue::glue("Grouping {grouping_number}: Formula will be: {cox_formula}"))

    cox <- coxph(as.formula(cox_formula), data = df)
        
    m_resid = residuals(cox, type = "martingale")

    if(null_model == TRUE) {
        final_df <- NULL
    } else {
        final_df <- broom::tidy(cox, exponentiate = TRUE, conf.int = TRUE) %>% 
            select(-statistic, -std.error) %>% 
            mutate(
                across(where(is.numeric), ~round(.x, 3))
            ) %>%
            select(term, estimate, conf.low, conf.high, p.value)   
    }


    return(list(cox, final_df, m_resid, df))
    
}

## Non-avoidable ED attendances

In [ ]:
cox_non_avoid_cohort_model <- cox_fn(groupinga_df, grouping_number, non_avoid = TRUE, basic = TRUE)

In [ ]:
cox_non_avoid_cohort_model[[1]]

## All ED attendances

In [ ]:
cox_all_cohort_model <- cox_fn(groupinga_df, grouping_number, non_avoid = FALSE, basic = TRUE)

In [ ]:
cox_all_cohort_model[[1]]

# Assessing Goodness-of-Fit using residuals
Chapter 7, Applied Survival Analysis Using R text (https://xsliulab.github.io/Workshop/2021/week3/survival-analysis-book.pdf)


Bradburn, M. J., Clark, T. G., Love, S. B., & Altman, D. G. (2003). Survival Analysis Part III: Multivariate data analysis – choosing a model and assessing its adequacy and fit. British Journal of Cancer, 89(4), 605–611. https://doi.org/10.1038/sj.bjc.6601120

# Martingale residuals
p88 has some info about why Martingale residuals are useful. In a nutshell: These residuals are symmetrically distributed with expected value 0 (if the fitted model is correct). This primarily is a test for non-linearity, I'm not sure they are useful.

In [ ]:
null_non_avoid_model <- cox_fn(df = groupinga_df, grouping = grouping_number, non_avoid = TRUE, null_model = TRUE)

In [ ]:
null_all_model <- cox_fn(df = groupinga_df, grouping = grouping_number, non_avoid = FALSE, null_model = TRUE)

In [ ]:
martingale_plot_distribution <- function(model, covar, transform = "none") {
    
    #print(glue::glue("Covar is {covar}"))    
    resid <- model[[3]]
    
    if(transform == "log") {
        resid <- log(resid)
    }

    df <- tibble(
        resid = resid,
        covar = model[[4]] %>% select(all_of(covar)) %>% pull()
    )
    
    return(df)
}

plot_martingale_resid_fn <- function(model, covar, transform = "none") {
    
    df <- martingale_plot_distribution(model, covar, transform)
    
    gplot <- df %>% ggplot(aes(x = covar, y = resid)) 

    
    if(is.character(df$covar) | is.factor(df$covar)) {
        
        gplot + geom_boxplot() +
            scale_x_discrete(name = covar)
        
    } else {
        
        points_data <- df
        if(nrow(df) > 10000) {
            points_data <- df %>% sample_n(8000)
        }
        
        gplot +
            geom_point(col = "grey", alpha = 0.1, data = points_data) +
            geom_smooth(col = "blue", se = TRUE, method = "locfit",
                                             linewidth = 1, alpha = 0.3) +
            scale_x_continuous(name = covar)
    }
    
}

plot_a_list <- function(master_list_with_plots, no_of_rows, no_of_cols) {

  patchwork::wrap_plots(master_list_with_plots, 
                        nrow = no_of_rows, ncol = no_of_cols)
}

martingale_plots <- function(model, non_avoid = TRUE) {
    
    options(repr.plot.width=14, repr.plot.height=6)
    #cols <- colnames(model[[4]])
    cols <- c("age")
    plots <- map(cols, plot_martingale_resid_fn, model = model)
    
    final_plot <- plot_a_list(plots, 1, 1)
    final_plot
}



## Non-avoidable ED attendance

In [ ]:
martingale_plots(null_non_avoid_model)

## All ED attendances

In [ ]:
martingale_plots(null_all_model)

# Influential values with dfbeta
Note using non-random effects model for all of this stuff for now...

In [ ]:
cox_non_avoid_model <- cox_fn(groupinga_df, grouping_number, non_avoid = TRUE, basic = FALSE)
cox_non_avoid <- cox_non_avoid_model[[1]]

cox_all_model <- cox_fn(groupinga_df, grouping_number, non_avoid = FALSE, basic = FALSE)
cox_all <- cox_all_model[[1]]

In [ ]:
ggcoxdiag <- function(
    fit, 
    type = c("martingale", "deviance", "score", "schoenfeld",
                               "dfbeta", "dfbetas", "scaledsch","partial"),
    linear.predictions = type %in% c("martingale", "deviance"),
    ox.scale = if_else(linear.predictions, "linear.predictions", "observation.id"),
    ...,
    hline = TRUE,
    sline = TRUE, 
    sline.se = TRUE,
    hline.col = "red", hline.size = 1, hline.alpha = 0.5, hline.yintercept = 0, hline.lty = 'dashed',
    sline.col = "blue", sline.size = 1, sline.alpha = 0.3, sline.lty = 'dashed',
    point.col = "black", point.size = 1, point.shape = 19, point.alpha = 0.3,
    points = FALSE,
    title = NULL, subtitle = NULL, caption = NULL,
    ggtheme = ggplot2::theme_bw(),
    df_only = FALSE
) {
    
    model <- fit
    type <- match.arg(type)
    res <- as.data.frame(resid(fit, type = type))
            
    if(type %in% c("martingale", "deviance")) col_names <- "residuals" else col_names <- names(stats::coef(fit))
    colnames(res) <- col_names    
    
    xlabel <- "The index number of observations"
    ylabel <- paste0("Residuals (type = ", type, ")" )
    
    switch(ox.scale,
         linear.predictions = {
           if (!(type %in% c("martingale", "deviance")))
             warning("ox.scale='linear.predictions' works only with type=martingale/deviance")
           xval <- predict(fit, type="lp")
           xlabel <- "Linear Predictions"
         },
         observation.id = {
           xval <- 1:nrow(res)
           xlabel <- "Observation Id"
         },
         time = {
           if (!(type %in% c("schoenfeld", "scaledsch")))
             warning("ox.scale='time' works only with type=schoenfeld/scaledsch")
           #xval <- as.numeric(rownames(res))
           xval <- as.numeric(substring(rownames(res),2))
           xlabel <- "Time"
         },
         {warning("ox.scale should be one of linear.predictions/observation.id/time")})

    res$xval <- xval
                      
    data2plot <- tidyr::pivot_longer(res, cols = all_of(col_names),
                              names_to = "covariate", values_to = "res")

    if(df_only) {
        return(data2plot)
    }
    
    tot_d2p <- nrow(data2plot)
    
    gplot <- ggplot(aes(xval, res), data = data2plot)
    
    if (points) {
        
        points_data <- data2plot
        if(tot_d2p > 10000) {
            points_data <- data2plot %>% sample_n(8000)
        }
        
        gplot <- gplot + geom_point(
            data = points_data,
            col = point.col, 
            shape = point.shape, 
            size = point.size, 
            alpha = point.alpha
        )
    }
    
    if (hline) gplot <- gplot + geom_hline(yintercept=hline.yintercept, col = hline.col,
                                         linewidth = hline.size, lty = hline.lty, alpha = hline.alpha)

    if (sline) gplot <- gplot + geom_smooth(col = sline.col, se = sline.se, method = "locfit",
                                         linewidth = sline.size, lty = sline.lty, alpha = sline.alpha)

    gplot <- gplot + labs(x = xlabel, y = ylabel, title = title, subtitle = subtitle, caption = caption) + ggtheme
    # customization
    gplot <- ggpubr::ggpar(gplot, ...)

    gplot <- gplot + facet_wrap(~covariate, scales = "free")
    gplot
    
}

## Non-avoidable ED attendances

In [ ]:
options(repr.plot.width=14, repr.plot.height=14)
cox_diag_non_avoid <- ggcoxdiag(fit = cox_non_avoid, type = "dfbeta", ox.scale = "observation.id", points = T)

In [ ]:
cox_diag_non_avoid

In [ ]:
# Let's remove outliers (2s.d.) and see what happens

In [ ]:
non_avoid_dfbeta_df <- ggcoxdiag(fit = cox_non_avoid, type = "dfbeta", ox.scale = "observation.id", points = T, df_only = TRUE)
non_avoid_dfbeta_df %>% glimpse()

In [ ]:
# https://cran.r-project.org/web/packages/olsrr/vignettes/influence_measures.html#:~:text=In%20general%2C%20large%20values%20of,as%20a%20size%2Dadjusted%20cutoff.
n_rows <- nrow(non_avoid_dfbeta_df)
non_avoid_dfbeta_df2 <- non_avoid_dfbeta_df %>% 
    group_by(covariate) %>% 
    mutate(
        two_sd = if_else((abs(res) <= 2*sd(res)), 1, 0),
        bkw = if_else((abs(res) <= 2/sqrt(n_rows)), 1, 0)
    ) %>% ungroup()

In [ ]:
# Get number of covariates that should be present:
n_cov <- non_avoid_dfbeta_df2 %>% count(covariate) %>% nrow()
non_avoid_dfbeta_df3 <- non_avoid_dfbeta_df2 %>% filter(two_sd == 1) %>% count(xval) %>% filter(n == n_cov)
non_avoid_dfbeta_df3a <- non_avoid_dfbeta_df2 %>% filter(bkw == 1) %>% count(xval) %>% filter(n == n_cov)

In [ ]:
groupinga_df %>% count()

In [ ]:
non_avoid_dfbeta_df3 %>% count()

In [ ]:
non_avoid_dfbeta_df3a %>% count()

In [ ]:
grouping_no_outliers_df <- groupinga_df %>% mutate(row_id = row_number()) %>% semi_join(non_avoid_dfbeta_df3a, by=c("row_id" = "xval")) %>% select(-row_id)
cox_non_avoid_no_outliers_model <- cox_fn(grouping_no_outliers_df, grouping_number, non_avoid = TRUE, basic = FALSE)
cox_non_avoid_no_outlier <- cox_non_avoid_no_outliers_model[[1]]

In [ ]:
cox_non_avoid_no_outlier_diag <- ggcoxdiag(fit = cox_non_avoid_no_outlier, type = "dfbeta", ox.scale = "observation.id", points = T)

In [ ]:
cox_non_avoid_no_outlier_diag

## All ED attendances

In [ ]:
ggcoxdiag(fit = cox_all, type = "dfbeta", ox.scale = "observation.id", points = T)

# Checking the proportional hazards assumption
From cox.zph: terms - if TRUE, do a test for each term in the model rather than for each separate covariate. For a factor variable with k levels, for instance, this would lead to a k-1 degree of freedom test. The plot for such variables will be a single curve evaluating the linear predictor over time.

## Non-avoidable ED attendances

In [ ]:
zph_non_avoid <- cox.zph(cox_non_avoid, terms = TRUE)
zph_non_avoid

In [ ]:
zph_non_avoid_dfbeta <- cox.zph(cox_non_avoid_no_outlier, terms = TRUE)
zph_non_avoid_dfbeta

## All ED attendances

In [ ]:
zph_all <- cox.zph(cox_all, terms = TRUE)
zph_all

In [ ]:
cox.zph(cox_all, terms = FALSE)

# Log-log plots

In [ ]:
#grouping_df %>% glimpse()

In [ ]:
ll_plots <- function(df, grouping_number, non_avoid) {

    cols <- c("cohort", "age", "age_cat", "sex", "ooh", "imd_quintile", "ethnicity_simple")
    
    plots <- map(cols, ~ km_plot_fn(covars = .x, df = df, grouping_num = grouping_number, non_avoid = non_avoid, type = "cloglog"))
    
    # Need map statement to get first item (the plot) from each list item)
    final_plot <- plot_a_list(map(plots, 1), length(cols), 1)
    final_plot
}

## Non-avoidable ED attendances

In [ ]:
zph_non_avoid

In [ ]:
options(repr.plot.width=14, repr.plot.height=40)
ll_plots(groupinga_df, grouping_number, "_non_avoid")

# All ED attendances

In [ ]:
zph_all

In [ ]:
options(repr.plot.width=14, repr.plot.height=40)
ll_plots(groupinga_df, grouping_number, "")

# Schoenfeld residuals

## Non-avoidable ED attendances

In [ ]:
options(repr.plot.width=14, repr.plot.height=20)
ggcoxzph(zph_non_avoid, resid = TRUE, point.alpha = 0.1, point.col = 'cadetblue3')

## All ED attendances

In [ ]:
ggcoxzph(zph_all, resid = TRUE, point.alpha = 0.1, point.col = 'cadetblue3')